In [1]:
import pandas as pd
import re
import torch
from torch import nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

from tqdm import tqdm
tqdm.pandas()

/home/marie/newenv/lib/python3.8/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix


In [2]:
df = pd.read_csv('../lab_1/rureviews/women-clothing-accessories.3-class.balanced.csv', encoding="utf8", sep='\t')

In [3]:
df = df[df['sentiment'] != 'neautral']

In [4]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
model.cuda() 

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29564, 312, padding_idx=0)
    (position_embeddings): Embedding(512, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [5]:
def embed_bert_cls(text):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [6]:
df['vector'] = df['review'].progress_apply(embed_bert_cls)

100%|██████████| 60000/60000 [01:32<00:00, 650.62it/s]


In [30]:
df

,review,sentiment,vector
0,качество плохое пошив ужасный (горловина напер...,negative,"[0.09496676, 0.019417698, -0.030974543, -0.052..."
1,"Товар отдали другому человеку, я не получила п...",negative,"[0.017277999, -0.020034766, -0.07157768, -0.04..."
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,"[0.058527857, 0.02429782, 0.019940346, -0.0296..."
3,"товар не пришел, продавец продлил защиту без м...",negative,"[-0.08016944, -0.064301595, -0.06941225, -0.03..."
4,"Кофточка голая синтетика, носить не возможно.",negative,"[0.0016684465, 0.0081783235, -0.02211897, -0.0..."
...,...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive,"[0.042270456, 0.01640827, -0.062187687, -0.059..."
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive,"[0.078745216, 0.080482885, -0.011663303, -0.04..."
89997,спасибо большое ) продовца рекомендую.. заказа...,positive,"[-0.0045187282, 0.05860255, -0.022774022, -0.0..."
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive,"[0.067568764, 0.043857623, -0.07193526, -0.035..."


In [110]:
X = [list(x) for x in df['vector'].values]
y = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [112]:
X_train = [torch.from_numpy(np.array(x)) for x in X_train]
X_test = [torch.from_numpy(np.array(x)) for x in X_test]

In [113]:
y_train = torch.from_numpy(np.array(y_train.values))
y_test = torch.from_numpy(np.array(y_test.values))

In [122]:
model = nn.Sequential(
            nn.Linear(312, 256),
            nn.ReLU(),
            nn.Linear(256, 2),
        )

In [123]:
optimizer = torch.optim.SGD([
                {'params': model.parameters()},
            ], lr=3e-2, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

In [124]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [125]:
model.to(device)

Sequential(
  (0): Linear(in_features=312, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=2, bias=True)
)

In [126]:
counter = 0
epochs = 3000
for i in range(0, epochs):
        X = torch.stack(X_train).to(device)
        y = y_train.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        

        print(loss.item())

0.6933491230010986
0.6932593584060669
0.6930902600288391
0.6928523182868958
0.6925550103187561
0.6922079920768738
0.691819429397583
0.691396176815033
0.6909441947937012
0.6904682517051697
0.6899723410606384
0.6894585490226746
0.6889289617538452
0.6883841753005981
0.6878252029418945
0.6872516870498657
0.6866629123687744
0.6860588192939758
0.6854379773139954
0.6847995519638062
0.6841433644294739
0.6834684014320374
0.6827739477157593
0.682059645652771
0.6813252568244934
0.6805710196495056
0.6797971725463867
0.6790035963058472
0.6781896352767944
0.6773543953895569
0.6764972805976868
0.6756166815757751
0.6747108697891235
0.6737786531448364
0.6728182435035706
0.6718277931213379
0.6708061695098877
0.6697520613670349
0.668663501739502
0.6675397753715515
0.6663802862167358
0.665183961391449
0.6639509797096252
0.6626802682876587
0.661371648311615
0.6600243449211121
0.6586378812789917
0.6572111248970032
0.6557437181472778
0.6542341113090515
0.6526817679405212
0.6510853171348572
0.6494441032409668

In [127]:
y_pred = model(torch.stack(X_test).to(device)).argmax(axis=1).cpu().numpy()

In [128]:
from sklearn.metrics import accuracy_score

In [129]:
accuracy_score(y_test, y_pred)

0.8819166666666667